In [227]:
# 1) import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import selenium

In [228]:
# 2) Create a path to chrome driver, 3) load the path into browser 4) initialize the browser
executable_path = {'executable_path': '/Users/Pariah/Downloads/chromedriver'}
browser = Browser('chrome', **executable_path, headless=True)

In [231]:
# 5) initialize the page parameters and the first site visit:
###############################################################
next_page = "https://www.frys.com/search?cat=-73060&nearbyStoreName=false&pType=pDisplay&rows=20&resultpage=0&start=0&rows=20"
page_remaining = True
page_no = 0
increment = 20
start_no = 0
# 6) make a variable that holds the url we want to scrape data from
laptops = {}

# 7) using the browser function, visit the webpage
browser.visit(next_page)
# 8) stop the scrape process for 1.25 seconds using the time function. This allows the webpage to load in full
time.sleep(1.25)

# 9) create the html variable that allows the browser to visit the HTML elements on the webpage
html = browser.html
# 10) initialize the BeautifulSoup module by telling it to parse html from the HTML elements on the webpage.
soup = bs(html, 'html.parser')


# 11) loop through until no products are found:
###############################################################
while page_remaining == True:
    
    # 12) Data retrieval logic: 
    ###############################################################
    try:
        mothers = soup.find_all('div', class_='togrid')
        for mother in mothers:
            title = mother.find('p', class_='productDescp').get_text().strip()
            price = mother.find('label', class_='red_txt').get_text().strip()
            partial_link = mother.find('p', class_='productDescp').find('a')['href'].strip()
            base_url = "https://www.frys.com/"
            link = base_url + partial_link
            
            results = mother.find_all('div', class_='prodModel')
            for result in results:
                new_list = result.find_all('p', class_='mar-btm')
                for p in new_list:
                    dataitem = p.text.replace(" ", "")
                    di = dataitem.replace('\n', "")
                    dataitem_split = di.split(":")
                    if dataitem_split[0] == "Brand":
                        brand = dataitem_split[1]
                    if dataitem_split[0] == "UPC":
                        upc = dataitem_split[1]
                    if dataitem_split[0] == "Model":
                        model = dataitem_split[1]
                        
            # 13) create a new dictionary that will hold the data that we scrape
            dic = {
                'title': title,
                'price': price,
                'link': link,
                'upc': upc,
                'model': model,
                'brand': brand
            }
            print(dic)
            
    except AttributeError:
        # errors out when there are 2 prices in red (price with rebate)
        print("error found. skipping...")
    
    
    # 14) then process the Next Page logic:
    ###############################################################
    page_no += 1
    start_no = start_no + increment
    next_page = f'https://www.frys.com/search?cat=-73060&nearbyStoreName=false&pType=pDisplay&rows=20&resultpage={page_no}&start={start_no}&rows=20'

    browser.visit(next_page)
    time.sleep(1.25)

    html = browser.html
    soup = bs(html, 'html.parser') 
    
    next_page_avail = soup.find_all('p')
    for p in next_page_avail:
        message = p.text
        if message == " No products were found that matched your search":
            page_remaining = False
        


{'title': 'Dell E6230 12.5" Refurbished Laptop With Intel Ci5-3320m Processor, 4GB Memory, 250GB Hard Drive, Windows 10', 'price': '$399.00', 'link': 'https://www.frys.com/product/9907916;jsessionid=09974825A051EA0B05F69A4BD9BEFE43.node3?site=sr:SEARCH:MAIN_RSLT_PG', 'brand': 'Dell', 'upc': '696897763632', 'model': 'E6230i5-3320m'}
{'title': 'HP PB 6470B 14" Refurbished Laptop With Intel Ci5-3210M Processor, 8GB Memory, 500GB Hard Drive, DVD-ROM, Windows 10', 'price': '$399.00', 'link': 'https://www.frys.com/product/9800005;jsessionid=09974825A051EA0B05F69A4BD9BEFE43.node3?site=sr:SEARCH:MAIN_RSLT_PG', 'brand': 'HP', 'upc': '696897751172', 'model': 'PB6470BCi5-3210M'}
{'title': 'Lenovo N21 11.6" Refurbished Chromebook With Intel Celeron N2840 Processor, 4GB Memory, 16GB Storage, Chrome OS', 'price': '$199.00', 'link': 'https://www.frys.com/product/9914956;jsessionid=09974825A051EA0B05F69A4BD9BEFE43.node3?site=sr:SEARCH:MAIN_RSLT_PG', 'brand': 'Lenovo', 'upc': '696897764257', 'model': '